In [184]:
import pickle
import numpy as np
import pandas as pd
from time import time
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import string
import collections
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint

 
BILL_PROGRESS = {}
BILL_PROGRESS[1] = "Introduced"
BILL_PROGRESS[2] = "Engrossed"
BILL_PROGRESS[3] = "Enrolled"
BILL_PROGRESS[4] = "Passed"
BILL_PROGRESS[5] = "Vetoed"
BILL_PROGRESS[6] = "Failed"
BILL_PROGRESS[7] = "Override"
BILL_PROGRESS[8] = "Chaptered"
BILL_PROGRESS[9] = "Refer"
BILL_PROGRESS[10] = "Report Pass"
BILL_PROGRESS[11] = "Report DNP"
BILL_PROGRESS[12] = "Draft"
"""
1 Introduced
2 Engrossed
3 Enrolled
4 Passed
5 Vetoed
6 Failed Limited support based on state
7 Override Progress array only
8 Chaptered Progress array only
9 Refer / Progress array only
10 Report Pass / Progress array only
11 Report DNP / Progress array only
12 Draft Progress / array only
"""

POLITICAL_PARTY = {}
POLITICAL_PARTY[1] = "Democrat"
POLITICAL_PARTY[2] = "Republican"
POLITICAL_PARTY[3] = "Independent"
POLITICAL_PARTY[4] = "Green Party"
POLITICAL_PARTY[5] = "Libertarian"
POLITICAL_PARTY[6] = "Nonpartisan"


TEXT_TYPES = {}
TEXT_TYPES[1] = "Introduced"
TEXT_TYPES[2] = "Committee Substitute"
TEXT_TYPES[3] = "Amended"
TEXT_TYPES[4] = "Engrossed"
TEXT_TYPES[5] = "Enrolled"
TEXT_TYPES[6] = "Chaptered"
TEXT_TYPES[7] = "Fiscal Note"
TEXT_TYPES[8] = "Analysis"
TEXT_TYPES[9] = "Draft"
TEXT_TYPES[10] = "Conference Substitu"
TEXT_TYPES[11] = "Prefiled"
TEXT_TYPES[12] = "Veto Message"
TEXT_TYPES[13] = "Veto Response"
TEXT_TYPES[14] = "Substitute"

BILL_TYPE = {}
BILL_TYPE[1] = "Bill"
BILL_TYPE[2] = "Resolution"
BILL_TYPE[3] = "Concurrent Resolution"
BILL_TYPE[4] = "Joint Resolution"
BILL_TYPE[5] = "Joint Resolution Constitutional Amendment"
BILL_TYPE[6] = "Executive Order"
BILL_TYPE[7] = "Constitutional Amendment"
BILL_TYPE[8] = "Memorial"
BILL_TYPE[9] = "Claim"
BILL_TYPE[10] = "Commendation"
BILL_TYPE[11] = "Committee Study Request"
BILL_TYPE[12] = "Joint Memorial"
BILL_TYPE[13] = "Proclamation"
BILL_TYPE[14] = "Study Request"
BILL_TYPE[15] = "Address"
BILL_TYPE[16] = "Concurrent Memorial"
BILL_TYPE[17] = "Initiative"
BILL_TYPE[18] = "Petition"
BILL_TYPE[19] = "Study Bill"
BILL_TYPE[20] = "Initiative Petition"
BILL_TYPE[21] = "Repeal Bill"
BILL_TYPE[22] = "Remonstration"
BILL_TYPE[23] = "Committee Bill"


"""
1 B Bill
2 R Resolution
3 CR Concurrent Resolution
4 JR Joint Resolution
5 JRCA Joint Resolution Constitutional Amendment
6 EO Executive Order
7 CA Constitutional Amendment
8 M Memorial
9 CL Claim
10 C Commendation
11 CSR Committee Study Request
12 JM Joint Memorial
13 P Proclamation
14 SR Study Request
15 A Address
16 CM Concurrent Memorial
17 I Initiative
18 PET Petition
19 SB Study Bill
20 IP Initiative Petition
21 RB Repeal Bill
22 RM Remonstration
23 CB Committee Bill
"""
VOTES_TYPE = {}

VOTES_TYPE[1] = "Yea"
VOTES_TYPE[2] = "Nay"
VOTES_TYPE[3] = "Not Voting / Abstain"
VOTES_TYPE[4] = "Absent / Excused"


MIME_TYPES = {}

MIME_TYPES[1] = "HTML"
MIME_TYPES[2] = "PDF"
MIME_TYPES[3] = "WordPerfect"
MIME_TYPES[4] = "MS Word (.doc)"
MIME_TYPES[5] = "Rich Text Format"
MIME_TYPES[6] = "MS word 2007 (.docx)"

    
with open('NH_bill_dict','rb') as f:
    bill_data = pickle.load(f)
with open('NH_vote_dict','rb') as f:
    vote_data = pickle.load(f)
with open('NH_people_dict','rb') as f:
    people_data = pickle.load(f)

with open('NH_people_dict','rb') as f:
    people_data = pickle.load(f)
    
with open('NHlegoverall.dict','rb') as f:
    leg_data = pickle.load(f)
    


    
all_bills = {}
all_bills['bills'] = []

for key in bill_data.keys():
    for k in bill_data[key]:
        all_bills['bills'].append(k)

all_votes = {}
all_votes['votes'] = []

for key in vote_data.keys():
    for k in vote_data[key]:
        all_votes['votes'].append(k)

from collections import defaultdict

def list_duplicates(seq):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        tally[item].append(i)
    return ((key,locs) for key,locs in tally.items() 
                            if len(locs)>1)


people_hash = {}
        
all_people = {}
all_people['people'] = []

for key in people_data.keys():
    for k in people_data[key]:
        all_people['people'].append(k)
        if not int(k['people_id']) in people_hash:
            people_hash[int(k['people_id'])] = int(k['party_id'])
        
print(len(all_people['people']))
print("hashtable" ,len(people_hash))

print("before", len(all_people['people']))
dups = []
party_dups = []
final_list = [] 
for num in all_people['people']: 
    if num['people_id'] not in final_list: 
        final_list.append(num['people_id'])
    else:
        dups.append(num['people_id'])
        party_dups.append(num['party'])
        
for dup in sorted(list_duplicates(dups)):
    tempy = []
    for dd in dup[1]:
        tempy.append(party_dups[dd])
    print(dup[0], tempy)

        
print("after", len(dups))
counter = 0
for d in dups:
    if counter < 10:
        for p in all_people['people']:
            if d == p['people_id']:
                print(p['people_id'], p['party'])
        counter += 1

"""
visited_index = []
visited = []
for i, val in enumerate(all_people['people']):
    if not int(val['people_id']) in visited:
        print(int(val['people_id']), "->", visited[visited.index(int(val['people_id']))])
        print(val['party'], "->", all_people['people'][visited.index(int(val['people_id']))]['party'])
    visited.append(val['people_id'])
    visited_index.append(i)
"""


        
bill_votes = {}
bill_votes['v'] = []
for key in vote_data.keys():
    if ((key == 2017) or (key == 2018) or (key == 2019)):
        for k in vote_data[key]:
            bill_votes['v'].append(k)
print(len(bill_votes['v']))









2132
hashtable 736
before 2132
4744 ['R', 'R', 'R']
4747 ['R', 'R', 'R', 'R']
4750 ['D', 'D', 'D', 'D']
4753 ['R', 'R']
4754 ['R', 'R', 'R', 'R']
4760 ['R', 'R', 'R', 'R']
4766 ['R', 'R', 'R', 'R']
4768 ['R', 'R', 'R']
4770 ['R', 'R', 'R', 'R']
4772 ['R', 'R', 'R', 'R']
4773 ['R', 'R', 'R']
4775 ['R', 'R', 'R', 'R']
4783 ['D', 'D', 'D', 'D']
4785 ['D', 'D', 'D', 'D']
4789 ['D', 'D', 'D', 'D']
4792 ['D', 'D', 'D']
4796 ['D', 'D', 'D']
4802 ['D', 'D', 'D', 'D']
4803 ['R', 'R', 'R', 'R']
4809 ['D', 'D', 'D', 'D']
4810 ['D', 'D', 'D', 'D']
4812 ['D', 'D', 'D', 'D']
4817 ['R', 'R', 'R', 'R']
4822 ['D', 'D', 'D', 'D']
4823 ['D', 'D', 'D', 'D']
4827 ['D', 'D', 'D']
4828 ['D', 'D', 'D', 'D']
4829 ['D', 'D', 'D', 'D']
4830 ['R', 'R', 'R']
4831 ['R', 'R', 'R']
4836 ['D', 'D', 'D', 'D']
4842 ['R', 'R', 'R', 'R']
4845 ['D', 'D', 'D', 'D']
4848 ['R', 'R', 'R']
4851 ['D', 'D', 'D']
4852 ['R', 'R', 'R']
4853 ['R', 'R', 'R', 'R']
4858 ['R', 'R', 'R']
4864 ['D', 'D', 'D', 'D']
4867 ['D', 'D', 'D']
4871

In [ ]:


bill_data_17_19 = {}
bill_data_17_19['bills'] = []

vote_data_17_19 = {}
vote_data_17_19['votes'] = []

passed_bills = []
vote_bill = []

for key in bill_data.keys():

    if key == 2017:
        print(key, len(bill_data[key]))
        for b in bill_data[key]:
            size = 0
            prog = b['progress']
            for p in prog:
                if p['event'] == 4:
                    passed_bills.append(b['bill_id'])
    elif key == 2018:
        print(key, len(bill_data[key]))
        for b in bill_data[key]:
            size = 0
            prog = b['progress']
            for p in prog:
                if p['event'] == 4:
                    passed_bills.append(b['bill_id'])

            

counter = 0
for key in bill_data.keys():
    if key == 2017 or 2018:
        for b in bill_data[key]:
            if b['completed'] == 1:
                counter += 1
print(counter)
print("bills passed from 2017-2019", len(passed_bills))
            
for b in passed_bills:
    if b == 1049105:
        print("TRUE")
            
for key in bill_data.keys():
    if key == 2017 or 2018 or 2019:
        for ele in bill_data[key]:
            if ele['votes'] == []:
                continue
            else:
                res = ele['votes']
                for r in res:
                    vote_bill.append(r['roll_call_id'])
            

                    
print(len(vote_bill))
poli_data = {}
poli_data['poli'] = []
#should be 1941 elements
for key in vote_data.keys():
    year = vote_data[key]
    for temp in year:
        if temp['roll_call_id'] in vote_bill:
            #print(temp['roll_call_id'])
            res = temp
            #print(res['bill_id'])
            val = {}
            val['bill_id'] = res['bill_id']
            val['roll_call_id'] = res['roll_call_id']
            val['date'] = res['date']
            val['yea'] = res['yea']
            val['nay'] = res['nay']
            val['nv'] = res['nv']
            val['absent'] = res['absent']
            val['total'] = res['total']
            val['passed'] = res['passed']
            val['chamber'] = res['chamber']
            val['votes'] = res['votes']
            poli_data['poli'].append(val)

print(poli_data['poli'][1]['votes'])






"""
for ele in res:
    bill_id = ele['bill_id']
    if ele['status'] == 4 or 3 or 2:
        bill_passed.append(bill_id)
        
        
    
print(len(bill_data_17_19['bills']))
print(len(bill_passed))


        
data = pd.read_pickle("NHLegTimeSeriesDS.DF")
data1 = pd.read_pickle("NHWinState.DF")
#print(data)
#print(data1)

range_list = {}
df = pd.concat([data, data1], axis=1, join='inner')

#print(res2)

df2 = df.loc[df['bill_id'].isin(list_of_bad_bills)]
df3 = df2.loc[df2['every stage passed'] == 1]

print(df3.bill_id.unique())


#[1136206 1158581 1173676 1178833 1158478]






count = 0
print(leg_data.keys())
for b in leg_data['bills']:
    if b['bill']['bill_id'] in list_of_bad_bills:
        print( b["bill"]['bill_id'])
"""




In [ ]:
with open('test.csv', mode='r') as infile:
    reader = csv.reader(infile)
    roll_call = {rows[0]:rows[1] for rows in reader}

with open('out_put_.csv', 'w') as f:
    f.write("%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s\n"%("Bill_id", "date", "people_id", "vote_id", "vote_text", "name", "party", "passed", "Chamber", "Total yea", "Total nay"))
    for key in roll_call.keys():
    	
    	res = legis.get_roll_call(roll_call[key])
    	_res = res['votes']
    	for __res in _res:
    		per = legis.get_sponsor(__res['people_id'])
        	f.write("%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s\n"%(key, str(res['date']), __res['people_id'], __res['vote_id'], str(__res['vote_text']), str(per['name']), str(per['party']), res['passed'], res['chamber'], res['yea'], res['nay']))


In [38]:
poli_dict = pd.read_csv('politician_data.csv')

print(poli_dict[poli_dict['people_id'] == 4775])

        Bill_id        date  people_id  vote_id vote_text           name  \
396     1173616  2019-03-14       4775        1       Yea  Sharon Carson   
820     1162256  2019-03-07       4775        2       Nay  Sharon Carson   
844     1136568  2019-05-15       4775        1       Yea  Sharon Carson   
1668     910055  2017-05-31       4775        2       Nay  Sharon Carson   
1691    1037818  2018-02-01       4775        1       Yea  Sharon Carson   
...         ...         ...        ...      ...       ...            ...   
148284  1173337  2019-03-14       4775        2       Nay  Sharon Carson   
149487   917001  2017-03-30       4775        1       Yea  Sharon Carson   
150673  1136244  2019-05-15       4775        2       Nay  Sharon Carson   
150697  1037865  2018-03-08       4775        1       Yea  Sharon Carson   
152694  1173428  2019-02-14       4775        2       Nay  Sharon Carson   

       party  passed Chamber  Total yea  Total nay  
396        R       1       S      

In [129]:
bills = pd.read_pickle("NHlegtext_embed_wBill.DF")
bills.loc[bills['bill_id'] == '895290']
subse = bill_votes['v'][1:5]
for b in subse:
    print(b['bill_id'])

895290
895290
894771
894791


In [185]:
bills = pd.read_pickle("NHlegtext_embed_wBill.DF")
print(bills)

print(bill_votes['v'][1])

data = {}

data['bill_id'] = []
data['roll_call_id'] = []
data['passed'] = []
data['chamber_id'] = []
data['people_id'] = []
data['vote_id'] = []



#print(bills.loc[bills['bill_id'] == 895290])
counter = 0
for i, val in bills.iterrows():
    
    if (not any(d['bill_id'] == int(val['bill_id']) for d in bill_votes['v'])):
        data['bill_id'].append(int(val['bill_id']))
        data['roll_call_id'].append(-1)
        data['passed'].append(-1)
        data['chamber_id'].append(-1)
        data['people_id'].append(-1)
        data['vote_id'].append(-1)
    else:
        for bb in bill_votes['v']:
            #tempy = bill_votes['v'][j]
            if bb['bill_id'] == int(val['bill_id']):
                    #print(val['bill_id'], "~~~>", bb['bill_id'])
                    #print(vote_temp)
                    for v in bb['votes']:
                        data['bill_id'].append(int(val['bill_id']))
                        data['roll_call_id'].append(bb['roll_call_id'])
                        data['passed'].append(bb['passed'])
                        data['chamber_id'].append(bb['chamber_id'])
                        
                        data['people_id'].append(people_hash[int(v['people_id'])] if int(v['people_id']) in people_hash else -1)
                        data['vote_id'].append(v['vote_id'])


        #counter += 1
print(len(data['bill_id']))
print(len(data['roll_call_id']))
print(len(data['passed']))
print(len(data['chamber_id']))
print(len(data['people_id']))
print(len(data['vote_id']))


poli_df = pd.DataFrame(data)

result = pd.merge(bills, poli_df, on='bill_id')

print(poli_df.loc[poli_df['bill_id'] == 895290])

print(result)
                    
                

       doc_id       dv1       dv2       dv3       dv4       dv5       dv6  \
0     1441765 -1.557997  0.141734 -0.438610 -0.451512 -0.996502 -0.442287   
1     1441965 -2.875763  0.328236 -0.462774 -0.697043 -1.127743  0.107598   
2     1441759 -2.216924 -0.002568  0.054848 -0.978794 -1.393849 -0.274661   
3     1441918 -2.157434 -0.300260  0.241523 -1.117743 -1.487071 -0.752131   
4     1441860 -2.526122 -0.334188  0.337743 -1.028891 -2.029144 -0.781107   
...       ...       ...       ...       ...       ...       ...       ...   
4055  1858525  0.269384  0.078609 -0.080544  1.339315 -0.418879 -0.722133   
4056  1978987  0.311415  0.039699 -0.291317  1.509251 -0.483186 -0.684144   
4057  1860172 -1.169458 -0.687591 -0.572571  0.955684  0.190207 -1.172075   
4058  2033756 -1.294992 -0.681913 -0.752976  1.021680  0.244887 -1.330637   
4059  1860514 -0.144612 -0.245435  0.821701  1.011145  1.022598  1.061322   

           dv7       dv8       dv9      dv10      dv11      dv12      dv13 

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
counter = 0
for i, val in poli_df.iterrows():
    if counter < 3:
        if (any(d['people_id'] == int(val['people_id']) for d in all_people['people'])):
            poli_df['party_id'] = d['party_id']
            print(d)
        counter += 1
print(poli_df)
        

KeyboardInterrupt: 